In [1]:
from torch.utils.data import DataLoader
from iterable_sql import SQLIterableDataset
import datetime

In [2]:
driver = 'ODBC+Driver+17+for+SQL+Server'
server = r'DESKTOP-5A37F25\SQLEXPRESS'
database = 'Test'
iterate_from = datetime.datetime(2020, 2, 10, 0, 0, 0, 0, tzinfo=datetime.timezone.utc)
iterate_to = datetime.datetime(2020, 2, 10, 23, 59, 59, 999999, tzinfo=datetime.timezone.utc)

In [9]:
args = {
    # Required parameters
    'conn_str': f'mssql://@{server}/{database}?driver={driver}',
    'table': 'Test.dbo.Tesla_Trades_and_Quotes_2020_2021',
    'iterable_column': 'date_time',
    # Optional parameters
    'iterate_from': iterate_from,
    'iterate_to': iterate_to,
    'columns': ['date_time', 'price', 'size'],
       }

In [10]:
# test object parameters
dataset = SQLIterableDataset(**args)
dataloader = DataLoader(dataset, batch_size=10000,
                        shuffle=False, drop_last=False
                        )
print(dataset.columns, end='\n\n')

['date_time', 'price', 'size']



In [11]:
%%time
# test print batches
from IPython.display import clear_output
dataset = SQLIterableDataset(**args)
dataloader = DataLoader(dataset, batch_size=5,
                        shuffle=False, drop_last=False,
                        collate_fn=dataset.sqlcollate)
for i, batch in enumerate(dataloader):
    if i == 5: break
    clear_output(wait=True)
    print(batch)

{'date_time': ['2020-02-10 09:00:31.244700+00:00', '2020-02-10 09:00:32.180800+00:00', '2020-02-10 09:00:34.024000+00:00', '2020-02-10 09:00:34.588800+00:00', '2020-02-10 09:00:34.705800+00:00'], 'price': tensor([755.0000, 755.0000, 755.0000, 754.9700, 754.0000], dtype=torch.float64), 'size': tensor([280,  30,  50,  12,  10])}
Wall time: 18.5 ms


In [13]:
%%time
# test work with tqdm
from tqdm.notebook import tqdm
dataset = SQLIterableDataset(**args)
dataloader = DataLoader(dataset, batch_size=15000,
                        shuffle=False, drop_last=False,
                        collate_fn=dataset.sqlcollate)
for batch in tqdm(dataloader):
    pass

0it [00:00, ?it/s]

Wall time: 10.1 s


In [15]:
%%time
# test load in pandas DataFrame
import pandas as pd
df = pd.DataFrame()
dataset = SQLIterableDataset(**args)
dataloader = DataLoader(dataset, batch_size=3000,
                        shuffle=False, drop_last=False,
                        collate_fn=dataset.sqlcollate)
for batch in dataloader:
    df = df.append(pd.DataFrame(batch), ignore_index=True)
    break
df

Wall time: 106 ms


,date_time,price,size
0,2020-02-10 09:00:00.077800+00:00,755.00,698
1,2020-02-10 09:00:00.416000+00:00,755.00,40
2,2020-02-10 09:00:00.509700+00:00,752.53,10
3,2020-02-10 09:00:00.892700+00:00,752.63,3
4,2020-02-10 09:00:05.960600+00:00,753.00,5
...,...,...,...
2995,2020-02-10 10:36:57.892400+00:00,789.00,30
2996,2020-02-10 10:36:57.892400+00:00,789.05,38
2997,2020-02-10 10:37:09.715200+00:00,788.90,15
2998,2020-02-10 10:37:14.596600+00:00,788.90,25
